In [4]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset
train_data_dir = r"D:\P_126\Dataset_new\train-20230326T152931Z-001\train"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Extract class names from the folder names
class_names = sorted(os.listdir(train_data_dir))

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Reshape the input data to include a time dimension
X_train_reshaped = []
y_train = []

for i in range(len(train_generator)):
    images, labels = train_generator[i]
    for j in range(len(images)):
        X_train_reshaped.append(images[j])
        y_train.append(labels[j])

X_train_reshaped = np.array(X_train_reshaped)
y_train = np.array(y_train, dtype=np.int32)

# Add a time dimension to the input data
X_train_reshaped = np.expand_dims(X_train_reshaped, axis=1)

# Define the DBLSTM model
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(1, img_width, img_height, 3)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(128, activation='relu', return_sequences=False))
model.add(Dense(len(class_names), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    X_train_reshaped,
    y_train,
    batch_size=batch_size,
    epochs=50)

# Save the trained model
#model.save('lung_detection_dblstm.h5')

Found 9971 images belonging to 6 classes.
Epoch 1/50
312/312 [==============================] - 212s 542ms/step - loss: 1.0277 - accuracy: 0.6095
Epoch 2/50
312/312 [==============================] - 167s 536ms/step - loss: 0.6442 - accuracy: 0.7623
Epoch 3/50
312/312 [==============================] - 167s 536ms/step - loss: 0.5415 - accuracy: 0.7973
Epoch 4/50
312/312 [==============================] - 169s 542ms/step - loss: 0.4400 - accuracy: 0.8413
Epoch 5/50
312/312 [==============================] - 166s 533ms/step - loss: 0.3534 - accuracy: 0.8705
Epoch 6/50
312/312 [==============================] - 165s 530ms/step - loss: 0.2951 - accuracy: 0.8936
Epoch 7/50
312/312 [==============================] - 163s 523ms/step - loss: 0.2211 - accuracy: 0.9244
Epoch 8/50
312/312 [==============================] - 165s 527ms/step - loss: 0.1764 - accuracy: 0.9364
Epoch 9/50
312/312 [==============================] - 163s 524ms/step - loss: 0.1300 - accuracy: 0.9526
Epoch 10/50
312/312 [=

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Predict probabilities for test data
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
class_labels = list(test_generator.class_indices.keys())

# Get true labels
true_labels = test_generator.classes

# Print classification report
print("Classification Report:")
print(classification_report(true_labels, y_pred, target_names=class_labels))

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, y_pred)

# Print confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

In [16]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load the saved model
model_path = 'lung_detection_dblstm.h5'
loaded_model = load_model(model_path)

# Define your custom class names
class_names = ['CARDIOMEGALY', 'COVID', 'NORMAL', 'PNEUMONIA', 'PNEUMOTHORAX', 'TUBERCULOSIS']

# Path to the image you want to predict
image_path = r"C:\Users\asuto\Desktop\intern ship 2\lung detection\archive\test-20230326T155708Z-001\test\COVID\COVID-19 (8).jpeg"

# Load and preprocess the image
img = image.load_img(image_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension
img_array = np.expand_dims(img_array, axis=1)  # Add a time dimension
img_array /= 255.0  # Rescale pixel values to [0, 1]

# Make a prediction
predictions = loaded_model.predict(img_array)
predicted_class = np.argmax(predictions)

# Get the predicted class name from the custom class names list
predicted_class_name = class_names[predicted_class]

# Display the predicted class
print(f"Predicted class: {predicted_class_name}")

1/1 [==============================] - 0s 363ms/step
Predicted class: COVID
